<a href="https://colab.research.google.com/github/adw1029/CSCI_544/blob/main/HW1_CSCI544.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade pip setuptools wheel

In [104]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [105]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz

## Read Data

In [106]:
import os
current_directory = os.getcwd()

In [107]:
print("Current Directory:", current_directory)


Current Directory: /content/drive/MyDrive/CSCI_544/hw1


In [108]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [109]:
google_drive_directory = '/content/drive/My Drive/CSCI_544/hw1'
os.chdir(google_drive_directory)

In [110]:
current_directory = os.getcwd()
print("Current Directory:", current_directory)

Current Directory: /content/drive/My Drive/CSCI_544/hw1


## Keep Reviews and Ratings

In [111]:
file_path = 'amz_data.tsv'

In [112]:
data = pd.read_csv(file_path, sep='\t', error_bad_lines=False, usecols=['review_body', 'star_rating'])

<ipython-input-112-8b1d537420a7>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(file_path, sep='\t', error_bad_lines=False, usecols=['review_body', 'star_rating'])
<ipython-input-112-8b1d537420a7>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep='\t', error_bad_lines=False, usecols=['review_body', 'star_rating'])


In [113]:
print(data)

        star_rating                                        review_body
0                 5                                     Great product.
1                 5  What's to say about this commodity item except...
2                 5    Haven't used yet, but I am sure I will like it.
3                 1  Although this was labeled as &#34;new&#34; the...
4                 4                    Gorgeous colors and easy to use
...             ...                                                ...
2640347           4  I can't live anymore whithout my Palm III. But...
2640348           4  Although the Palm Pilot is thin and compact it...
2640349           4  This book had a lot of great content without b...
2640350           5  I am teaching a course in Excel and am using t...
2640351           5  A very comprehensive layout of exactly how Vis...

[2640352 rows x 2 columns]


Separate out review and rating, dropped null values

In [114]:
data['star_rating'] = pd.to_numeric(data['star_rating'], errors='coerce')

In [115]:
data['star_rating'].dtypes

dtype('float64')

### Statistics of three classes

In [116]:
print("Number of rating more than 3:", len(data[data['star_rating'] > 3]))
print("Number of rating less or equal to 2:", len(data[data['star_rating'] <= 2]))
print("Number of rating = 3 (Discard):", len(data[data['star_rating'] == 3]))

Number of rating more than 3: 2001258
Number of rating less or equal to 2: 445383
Number of rating = 3 (Discard): 193694


In [117]:
df_RevRat = data[data['star_rating'] != 3]

In [118]:
contains_null = df_RevRat[['star_rating', 'review_body']].isnull().any()
contains_null

star_rating    True
review_body    True
dtype: bool

In [119]:
df_RevRat = data.dropna(subset=['star_rating', 'review_body'])

In [120]:
df_RevRat

,star_rating,review_body
0,5.0,Great product.
1,5.0,What's to say about this commodity item except...
2,5.0,"Haven't used yet, but I am sure I will like it."
3,1.0,Although this was labeled as &#34;new&#34; the...
4,4.0,Gorgeous colors and easy to use
...,...,...
2640347,4.0,I can't live anymore whithout my Palm III. But...
2640348,4.0,Although the Palm Pilot is thin and compact it...
2640349,4.0,This book had a lot of great content without b...
2640350,5.0,I am teaching a course in Excel and am using t...


In [121]:
df_RevRat['Sentiment'] = df_RevRat['star_rating'].apply(lambda x: 1 if x > 3 else 0)

<ipython-input-121-940d36b9d541>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RevRat['Sentiment'] = df_RevRat['star_rating'].apply(lambda x: 1 if x > 3 else 0)


In [122]:
df_RevRat['Sentiment']

0          1
1          1
2          1
3          0
4          1
          ..
2640347    1
2640348    1
2640349    1
2640350    1
2640351    1
Name: Sentiment, Length: 2640255, dtype: int64

check if there's null values

In [123]:
contains_null = df_RevRat[['star_rating', 'review_body']].isnull().any()
contains_null

star_rating    False
review_body    False
dtype: bool

### 20000 samples from positive (100000) and negative (100000) reviews

In [124]:
pos_rev = df_RevRat[df_RevRat['Sentiment'] == 1].sample(100000, random_state=42)
neg_rev = df_RevRat[df_RevRat['Sentiment'] == 0].sample(100000, random_state=42)
df_small = pd.concat([pos_rev, neg_rev])

In [125]:
df_small.describe()

,star_rating,Sentiment
count,200000.00000,200000.000000
mean,3.30734,0.500000
std,1.63064,0.500001
min,1.00000,0.000000
25%,2.00000,0.000000
50%,3.50000,0.500000
75%,5.00000,1.000000
max,5.00000,1.000000


In [126]:
df_small.sample(3)

,star_rating,review_body,Sentiment
1743794,5.0,We bought this extra extension phone for our d...,1
504708,5.0,"Gave as a gift, good quality",1
1662809,3.0,This pen is tough for me to rate. My husband ...,0


# Data Cleaning



### avg review length before cleaning

In [127]:
avglen_before = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_before)

Average text length: 315.554375


reviews to lower case

In [128]:
df_small['review_body'] = df_small['review_body'].str.lower()

remove URL and HTML

In [129]:
url_pattern = re.compile(r'https?://\S+|www\.\S+')

In [130]:
def remove_urls(text):
    return url_pattern.sub(r'', text)

# Apply the functions to the review column
df_small['review_body'] = df_small['review_body'].apply(remove_urls)

In [131]:
df_small['review_body'] = df_small['review_body'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())

<ipython-input-131-96fa6b4f62c6>:1: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df_small['review_body'] = df_small['review_body'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())


remove non-alpha

In [132]:
df_small['review_body'] = df_small['review_body'].str.replace('[^a-zA-Z]', ' ', regex=True)

remove extra spaces

In [133]:
df_small['review_body'] = df_small['review_body'].str.replace(r'\s+', ' ', regex=True)

contractions

In [134]:
pip install contractions

In [135]:
import contractions

In [136]:
df_small['review_body'] = df_small['review_body'].apply(contractions.fix)

###average review length after cleaning

In [137]:
avglen_after = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_after)

Average text length: 300.41889


# Pre-processing

In [138]:
from nltk.corpus import stopwords



```
# This is formatted as code
```

### 3 sample reviews before pre-processing / Avg length

In [139]:
print(df_small['review_body'].head(3))

386895     i reviewed the packaging previously and gave s...
2514581    i got tired of searching my place for envelope...
785896               the best faber castell never dissapoint
Name: review_body, dtype: object


### average review length before preprocessing

In [140]:
avglen_before = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_before)

Average text length: 300.41889


### remove the stop words

In [141]:
nltk.download('stopwords')

stp_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [142]:
df_small['review_body'] = df_small['review_body'].apply(lambda x: ' '.join([word for word in str(x).split() if word.lower() not in stp_words]))

## perform lemmatization  

In [143]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet

In [144]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [145]:
lemmatizer = WordNetLemmatizer()

In [146]:
get_wordnet_pos = lambda pos: wordnet.ADJ if pos.startswith('J')  else wordnet.VERB if pos.startswith('V') else wordnet.NOUN if pos.startswith('N') else wordnet.ADV if pos.startswith('R') else wordnet.NOUN

In [147]:
!pip install tqdm

In [148]:
from tqdm import tqdm
tqdm.pandas()

In [149]:
df_small['review_body'] = df_small['review_body'].progress_apply(
    lambda x: ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tag(word_tokenize(x))])
    )

100%|██████████| 200000/200000 [11:38<00:00, 286.21it/s]


In [150]:
print(df_small['review_body'].head(3))

386895     review packaging previously give star replaced...
2514581    got tire search place envelope decide buy grea...
785896                   best faber castell never dissapoint
Name: review_body, dtype: object


### average review length after  preprocessing

In [151]:
avglen_after = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_after)

Average text length: 178.07475


# TF-IDF Feature Extraction

#split testing (20%) and training (80%) set


In [152]:
vectorizer = TfidfVectorizer(max_features = 10000)
tfidf_mat = vectorizer.fit_transform(df_small['review_body'])

In [153]:
tfidf_mat

<200000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 4443301 stored elements in Compressed Sparse Row format>

# Perceptron

In [154]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [155]:
X = tfidf_mat
y = df_small['Sentiment']

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [157]:
perceptron = Perceptron(random_state= 42, eta0=1.0).fit(X_train, y_train)

In [158]:
prec_train = perceptron.predict(X_train)
prec_test = perceptron.predict(X_test)

In [159]:
# Training Metrics
perc_train_acc = accuracy_score(y_train, prec_train)
perc_train_f1 = f1_score(y_train, prec_train, average = 'weighted')
perc_train_pre = precision_score(y_train, prec_train, average = 'weighted')
perc_train_recall = recall_score(y_train, prec_train, average = 'weighted')

# Testing Metrics
perc_test_acc = accuracy_score(y_test, prec_test)
perc_test_f1 = f1_score(y_test, prec_test, average = 'weighted')
perc_test_precision = precision_score(y_test, prec_test, average = 'weighted')
perc_test_recall = recall_score(y_test, prec_test, average = 'weighted')

In [160]:
print('Result for Perceptron')
print('Training Accuracy Score :', perc_train_acc)
print('Training F1 Score :', perc_train_f1)
print('Training Precision Score :',perc_train_pre)
print('Training Recall Score :', perc_train_recall)

print('\nTesting Accuracy Score :', perc_test_acc)
print('Testing F1 Score :', perc_test_f1)
print('Testing Precision Score :', perc_test_precision)
print('Testing Recall Score :', perc_test_recall)

Result for Perceptron
Training Accuracy Score : 0.81055
Training F1 Score : 0.8104508698117924
Training Precision Score : 0.811197760460385
Training Recall Score : 0.81055

Testing Accuracy Score : 0.785375
Testing F1 Score : 0.7852780469090676
Testing Precision Score : 0.7859031562791542
Testing Recall Score : 0.785375


# SVM

In [161]:
from sklearn.datasets import make_classification
from sklearn import svm
from sklearn.linear_model import SGDClassifier

In [162]:
svm_sgd = SGDClassifier(loss='hinge', alpha = 0.001, max_iter = 1000, random_state=42)

In [163]:
svm_sgd.fit(X_train, y_train)

SGDClassifier(alpha=0.001, random_state=42)

In [164]:
SVM_train = svm_sgd.predict(X_train)
SVM_test = svm_sgd.predict(X_test)

In [165]:
# Training Metrics
svm_train_acc = accuracy_score(y_train, SVM_train)
svm_train_f1 = f1_score(y_train, SVM_train, average = 'weighted')
svm_train_pre = precision_score(y_train, SVM_train, average = 'weighted')
svm_train_recall = recall_score(y_train, SVM_train, average = 'weighted')

# Testing Metrics
svm_test_acc = accuracy_score(y_test, SVM_test)
svm_test_f1 = f1_score(y_test, SVM_test, average = 'weighted')
svm_test_precision = precision_score(y_test, SVM_test, average = 'weighted')
svm_test_recall = recall_score(y_test, SVM_test, average = 'weighted')

In [166]:
print('Result for SVM')
print('Training Accuracy Score :', svm_train_acc)
print('Training F1 Score :', svm_train_f1)
print('Training Precision Score :',svm_train_pre)
print('Training Recall Score :', svm_train_recall)

print('\nTesting Accuracy Score :', svm_test_acc)
print('Testing F1 Score :', svm_test_f1)
print('Testing Precision Score :', svm_test_precision)
print('Testing Recall Score :', svm_test_recall)

Result for SVM
Training Accuracy Score : 0.8176875
Training F1 Score : 0.8170008310926438
Training Precision Score : 0.8225373329844377
Training Recall Score : 0.8176875

Testing Accuracy Score : 0.817725
Testing F1 Score : 0.81693840800556
Testing Precision Score : 0.8232431885001974
Testing Recall Score : 0.817725


# Logistic Regression

In [167]:
from sklearn.linear_model import LogisticRegression


In [168]:
logReg = LogisticRegression(random_state=42, max_iter=1000)

In [169]:
logReg.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [170]:
log_train = logReg.predict(X_train)
log_test = logReg.predict(X_test)

In [171]:
# Training Metrics
log_train_acc = accuracy_score(y_train, log_train)
log_train_f1 = f1_score(y_train, log_train, average = 'weighted')
log_train_pre = precision_score(y_train, log_train, average = 'weighted')
log_train_recall = recall_score(y_train, log_train, average = 'weighted')

# Testing Metrics
log_test_acc = accuracy_score(y_test, log_test)
log_test_f1 = f1_score(y_test, log_test, average = 'weighted')
log_test_precision = precision_score(y_test, log_test, average = 'weighted')
log_test_recall = recall_score(y_test, log_test, average = 'weighted')

In [172]:
print('Result for Logistic Regression')
print('Training Accuracy Score :', log_train_acc)
print('Training F1 Score :', log_train_f1)
print('Training Precision Score :',log_train_pre)
print('Training Recall Score :', log_train_recall)

print('\nTesting Accuracy Score :', log_test_acc)
print('Testing F1 Score :', log_test_f1)
print('Testing Precision Score :', log_test_precision)
print('Testing Recall Score :', log_test_recall)

Result for Logistic Regression
Training Accuracy Score : 0.861625
Training F1 Score : 0.861622221418528
Training Precision Score : 0.8616547299553661
Training Recall Score : 0.861625

Testing Accuracy Score : 0.8479
Testing F1 Score : 0.8478866352751037
Testing Precision Score : 0.8480168465788752
Testing Recall Score : 0.8479


# Naive Bayes

In [173]:
from sklearn.naive_bayes import MultinomialNB

In [174]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB()

In [175]:
nb_train = nb.predict(X_train)
nb_test = nb.predict(X_test)

In [176]:
# Training Metrics
nb_train_acc = accuracy_score(y_train, nb_train)
nb_train_f1 = f1_score(y_train, nb_train, average = 'weighted')
nb_train_pre = precision_score(y_train, nb_train, average = 'weighted')
nb_train_recall = recall_score(y_train, nb_train, average = 'weighted')

# Testing Metrics
nb_test_acc = accuracy_score(y_test, nb_test)
nb_test_f1 = f1_score(y_test, nb_test, average = 'weighted')
nb_test_precision = precision_score(y_test, nb_test, average = 'weighted')
nb_test_recall = recall_score(y_test, nb_test, average = 'weighted')

In [177]:
print('Result for Naive Bayes')
print('Training Accuracy Score :', nb_train_acc)
print('Training F1 Score :', nb_train_f1)
print('Training Precision Score :',nb_train_pre)
print('Training Recall Score :', nb_train_recall)

print('\nTesting Accuracy Score :', nb_test_acc)
print('Testing F1 Score :', nb_test_f1)
print('Testing Precision Score :', nb_test_precision)
print('Testing Recall Score :', nb_test_recall)

Result for Naive Bayes
Training Accuracy Score : 0.82894375
Training F1 Score : 0.8289413608245326
Training Precision Score : 0.8289615925816899
Training Recall Score : 0.82894375

Testing Accuracy Score : 0.81885
Testing F1 Score : 0.8188498967444068
Testing Precision Score : 0.8188503830619202
Testing Recall Score : 0.81885
